In [6]:
import pandas as pd
from paths import CLEANED_DATA_DIR
from datetime import datetime
from data_split import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import catboost

In [8]:
df = pd.read_parquet(CLEANED_DATA_DIR / 'tabular_data.parquet')

In [2]:
X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 11, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(72312, 674)
y_train.shape=(72312,)
X_test.shape=(15982, 674)
y_test.shape=(15982,)


In [3]:
# use only past rides data
past_rides_columns = [c for c in X_train.columns if c.startswith('rides_')]
X_train_only_numeric = X_train[past_rides_columns]

In [4]:
model = catboost.CatBoostRegressor()
model.fit(X_train_only_numeric, y_train)

Learning rate set to 0.080526
0:	learn: 33.8928774	total: 93ms	remaining: 1m 32s
1:	learn: 31.5682556	total: 124ms	remaining: 1m 1s
2:	learn: 29.4043623	total: 152ms	remaining: 50.4s
3:	learn: 27.4780644	total: 181ms	remaining: 45.1s
4:	learn: 25.6629366	total: 210ms	remaining: 41.8s
5:	learn: 23.9648992	total: 243ms	remaining: 40.2s
6:	learn: 22.4346051	total: 272ms	remaining: 38.6s
7:	learn: 21.0146137	total: 301ms	remaining: 37.3s
8:	learn: 19.7115364	total: 333ms	remaining: 36.7s
9:	learn: 18.5130438	total: 362ms	remaining: 35.9s
10:	learn: 17.4469231	total: 394ms	remaining: 35.4s
11:	learn: 16.4414034	total: 424ms	remaining: 34.9s
12:	learn: 15.5425368	total: 472ms	remaining: 35.9s
13:	learn: 14.6972205	total: 510ms	remaining: 35.9s
14:	learn: 13.9494912	total: 550ms	remaining: 36.1s
15:	learn: 13.2600203	total: 584ms	remaining: 35.9s
16:	learn: 12.6285635	total: 614ms	remaining: 35.5s
17:	learn: 12.0735072	total: 652ms	remaining: 35.6s
18:	learn: 11.5651054	total: 692ms	remaining

In [5]:
X_test_only_numeric = X_test[past_rides_columns]
predictions = model.predict(X_test_only_numeric)
predictions

test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=2.6422
